In [ ]:
"""This script focuses on extracting the edits from a Google Doc and placing all of the
data associated with those edits, including the editor's name, 
the time the edit was made, the document the edit was made on """
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install oauth2client
!pip install pandas
!pip install regex
!pip install gspread
!pip install pytz

In [ ]:
'''This cell executes the authentication process
Pre requisites: token.json and client_secret.json from google cloud platform
Before running locally ->  pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
Copy the content of the cell and run it on your local machine to get token.json file
token expires after a week, delete the previous token and run the local machine script again to get a new token
Go to google cloud platform and obtain client_Secret.json file
Upload both the files in the runtime environment and run this cell'''

from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import pytz
from datetime import datetime, timezone
import time
import pandas as pd
import re
import os

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/drive", 'https://www.googleapis.com/auth/drive.activity', 
          'https://www.googleapis.com/auth/contacts', 'https://www.googleapis.com/auth/userinfo.profile'] 

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'client_secret.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())


In [ ]:
#This cell sets up the services required
"""
1. Following services can get number of revisions made to an item in the drive
2. Get number of edits made to an item in drive or shared drive
3. Reference file metadata to access contacts and get info about people who edited an item
"""

#Drive service to access revisions of a file
drive_service = build('drive', 'v3', credentials=creds)

#Service to get activity on a drive item and get metadata like timestamps, resourceName, filenames, etc
drive_activity_service = build('driveactivity', 'v2', credentials=creds)

#Access google account contacts using resourceName to get names, email address of the modifier.  
people_service = build('people', 'v1', credentials=creds)


In [ ]:
#this cell creates and maintains users list
users_list = {}
def get_user_name(resource_name):
  """
     This function takes a person's resource name and puts its corresponding
     details into a hashmap 
    
    Args: 
      resource_name(str) - resource name of the person who made an edit to the document

    Returns: 
      void function. writes people/id: name to hashmap
    
    Considerations:
      To get name of the person who made an edit, the person must be added to your 
      google contacts.
    
    Side Effects:
      Can only write details of a resource id if the said person is a contact in
      google workspace
  """
  global users_list
  if resource_name not in users_list:
    user = people_service.people().get(resourceName=resource_name,personFields='names').execute()
    user = user[ 'names'][0]['displayName']
    users_list[resource_name] = user
  return

In [ ]:
#This cell gets edits on a drive item and saves it to a CSV file

def get_edits_info(file_id, team_name):
  """
     This function takes the item id of a Google document 
     and fetches it's edits information
    
    Args: 
      file_id(str) - Id of the file for which edits are to be determined
      filename(str) - csv file to which we want to write data

    Returns: 
      Function is a void function. It just writes the dataframe to a csv file
    
    
    Side Effects:
      The function writes to the csv file in append mode. There could be duplicates
      if the function gets the same file id multiple times. 
  """
  global users_list
  time.sleep(5)
  drive_activity_results = drive_activity_service.activity().query(
    body={"pageSize": 1000,"itemName": f"items/{file_id}"}).execute()

  activities = drive_activity_results.get('activities', [])
  filename = []
  users = []
  mod_at = []
  team = []
  count = 1
  for activ in activities:
    resource_name = activ['actors'][0]['user']['knownUser']['personName']
    get_user_name(resource_name)
  for activity in activities:
      try:
          est = pytz.timezone('US/Eastern')
          utc = pytz.utc
          fmt = '%Y-%m-%d %H:%M:%S.%f'
          timestamp = activity['timestamp'][:-1]
          formatted_timestamp = datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%S.%f')
          utc_time =formatted_timestamp.replace(tzinfo=timezone.utc)
          est_time = utc_time.astimezone(est).strftime(fmt)
          file_name = activity['targets'][0]['driveItem']['title']
          resource_name = activity['actors'][0]['user']['knownUser']['personName']
          user = users_list[resource_name]
          filename.append(file_name)
          users.append(user)
          mod_at.append(est_time)
          team.append(team_name)
      except:
          print(f"{count} record(s) skipped for team: {team_name} and file: {filename[0]}")
          count +=1
 
  print(f"Writing {filename[0]} data to {team_name}.csv...")
  data = { "Team Name": team, "File": filename, "Modified By": users, "Modified at": mod_at}
  df = pd.DataFrame(data)
  df.to_csv(f"{team_name}.csv", sep=',', mode="a", index=False)
  print(f"{filename[0]} data written to {team_name}.csv...")
  return

In [ ]:
#This cell sets up 4 methods
#1. gets folder's id from the folder url
#2. gets item id from a specific folder using the folder url and filename
#3. get file ids of all the files in a specified folder
#4. get ids of child folders in a parent folder

def get_folder_id(url):
  """This function takes the URL of a Google Docs folder and 
    generates the folder's ID.
    
    Args: 
      url(str) - The folder URL
    
    Returns: 
      The folder's ID, which can be used to uniquely identify
        the Google Drive folder in question.
    
    Side Effects:
      Creates a search variable for regex to locate the file ID."""

  search = '/folders/\S*'
  try:
    folder_id = re.findall(search, url)
    folder_id = folder_id[0]
    folder_id = folder_id[9:]
    return folder_id
  except:
    return None

def get_id_by_name(url, filenames):
  """This function takes the URL of a Google Docs folder along with the name of 
     a file and generates the file's id.
    
    Args: 
      url(str) - The folder URL
      filenames(list) - list of filenames to be identified
    
    Returns: 
      The file IDs list, which can be used to uniquely identify the file
        within the Google Drive folder in question.
    
    Side Effects:
      returns None if exact match is not found"""

  folder_id = get_folder_id(url)
  file_ids = set()
  try:
    for filename in filenames:
      file_metadata = drive_service.files().list(q=f"'{folder_id}' in parents and name = '{filename}'", 
      fields='nextPageToken, files(id, name)').execute()
      file_ids.add(file_metadata['files'][0]['id'])
    return file_ids
  except:
    return None

def get_file_list(url):
  """This function takes the URL of a Google Docs folder and finds ids of all the files in it.
    
    Args: 
      url(str) - The folder URL
    
    Returns: 
      Set of file ids, within the Google Drive folder in question.
    
    Side Effects:
      Will identify only google document type files"""
  folder_id = get_folder_id(url)
  file_ids = set()
  try:
    google_docs = drive_service.files().list(q=f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.document'").execute()
    google_docs = google_docs['files']
    for google_doc in google_docs:
      file_ids.add(google_doc['id'])
    
    return file_ids
  except:
    return None

def get_child_folders(url):
  """This function takes the URL of a Google Docs folder and finds ids of all the folders in it.
    
    Args: 
      url(str) - The folder URL
    
    Returns: 
      Set of folder ids, within the Google Drive folder in question.
    
    Side Effects:
      Will identify only google folder type items"""
  folder_id = get_folder_id(url)
  folders_list = set()
  try:
    folders_metadata = drive_service.files().list(q=f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.folder'").execute()
    folders_metadata  = folders_metadata['files']
    for folder_metadata  in folders_metadata :
      folders_list.add((folder_metadata['name'],f"/folders/{folder_metadata['id']}"))
    return folders_list
  except:
    return None


In [ ]:
#This cell gets child folders of a parent folder
#Cell gets information of the files that are listed in the file_names list
#team folder url: https://drive.google.com/drive/u/0/folders/1PiYcfs3x1SXlKaSZ-2DenRxX0YuMG3wH
import time
file_names = ["Day 1 Reflection X", "Day 2 Reflection", "Day 3 Reflection"]

folder_url = input("Paste folder url here: ")

#each entry in the set consists of a tuple of folder name and folder ID
folder_list = list(get_child_folders(folder_url))
# count = 0
folder_list.sort()
for x in folder_list:
  print(x[0])
  # count+=1

Paste folder url here: https://drive.google.com/drive/u/0/folders/1PiYcfs3x1SXlKaSZ-2DenRxX0YuMG3wH
Team ALPHA
Team ALPHA Prime
Team BRAVO
Team CHARLIE
Team DUO
Team ECHO
Team FOX-TROT
Team GOLF
Team HOTEL
Team JULIET
Team KILO
Team LIMA
Team MIKE
Team NOVEMBER
Team OSCAR
Team PAPPA
Team QUEBEC
Team ROMEO
Team SIERRA
Team TANGO
Team UNIFORM
Team UNO
Team VICTOR
Team WHISKEY
Team X-RAY
Team YANKEE
Team ZULU
Y-Files


In [ ]:
# after we get the folder list, 
# we can iterate over the list and get all the files within each folder
for folder_name in folder_list:
  it_ids = get_id_by_name(folder_name[1], file_names)
  for it_id in it_ids:
    get_edits_info(it_id, folder_name[0])

In [ ]:
# after we get the folder list, 
# we can iterate over the list manually by changing the index and get all the files within each folder
#Run the cell only if you want to get edits of a specific file
try:
  folder_id = 0 #choose index that corresponds to a folder name in above cell
  item_ids = get_id_by_name(folder_list[folder_id][1], file_names)
  for item_id in item_ids:
    get_edits_info(item_id, folder_list[folder_id][0])
except:
  print("issue with folder_id", folder_id)
  print("issue with item ids", item_ids)
  print("issue with item id", item_id)

Writing Day 2 Reflection data to Team ALPHA.csv...
Day 2 Reflection data written to Team ALPHA.csv...
Writing Day 1 Reflection X data to Team ALPHA.csv...
Day 1 Reflection X data written to Team ALPHA.csv...
1 record(s) skipped for team: Team ALPHA and file: Day 3 Reflection
Writing Day 3 Reflection data to Team ALPHA.csv...
Day 3 Reflection data written to Team ALPHA.csv...


In [ ]:
#combine all the csv files to a single filename of your choice

filename = input("input a filename: ")

try:
  !cat *.csv >{filename}.csv
  print(f"{filename}.csv created successfully")
except:
  print("File merge error!")

input a filename: TeamAReflection
cat: '*.csv': No such file or directory
TeamAReflection.csv created successfully
